# Basic implementation of MCTS to Classifier Chains

### Generate some data

In [39]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

n_samples = 10000
n_features = 20
n_classes = 10
n_labels = 2
random_state = 0

X, Y = make_multilabel_classification(
    n_samples = n_samples,
    n_features = n_features,
    n_classes = n_classes,
    n_labels = n_labels,
    random_state = random_state)

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression

solver = "liblinear"
base = LogisticRegression(solver=solver)
chain = ClassifierChain(base)

chain = chain.fit(X_train,Y_train)

## Loss comparison

In [48]:
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.constraints import Constraint

constraint = Constraint(time=True, d_time=1.)
config1 = MCTSConfig(n_classes, selection_policy=UCB(), constraint=constraint, verbose=True, step_once=True, )
config2 = MCTSConfig(n_classes, constraint=constraint, verbose=True, step_once=True, )

In [49]:
M: int = 100
out1 = MCTS(chain, X_test[:M], config1)
out2 = MCTS(chain, X_test[:M], config2)

from sklearn.metrics import hamming_loss, zero_one_loss

hamming_loss(out1, Y_test[:M]), hamming_loss(chain.predict(X_test[:M]), Y_test[:M]), hamming_loss(out2, Y_test[:M])

100%|█████████████████████████████████████████| 100/100 [02:11<00:00,  1.31s/it]


(0.149, 0.147, 0.249)

# Implementation seems fine, now let's compare accuracies

In [ ]:
from sklearn.metrics import hamming_loss, zero_one_loss

secs_lis = [0.1, 0.5, 1.] #  2.]
M = min(100,len(Y_test))

hl_mt = []
hl_ct = []
hl_mc = []

zo_mt = []
zo_ct = []
zo_mc = []

y_chain = chain.predict(X_test[:M])
for secs in secs_lis:
#     continue
    y_mcts = []

    for i in trange(M, desc=f"MCTS Inference Constraint={secs}s", unit="it",colour="green"):
        y_mcts.append(MCTS(chain,X_test[i],secs=secs))

    y_mcts = np.array(y_mcts)
    
    hl_mt.append(hamming_loss(y_mcts,Y_test[:M]))
    hl_ct.append(hamming_loss(y_chain,Y_test[:M]))
    hl_mc.append(hamming_loss(y_chain,y_mcts))

    zo_mt.append(zero_one_loss(y_mcts,Y_test[:M]))
    zo_ct.append(zero_one_loss(y_chain,Y_test[:M]))
    zo_mc.append(zero_one_loss(y_chain,y_mcts))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(secs_lis,hl_mt,label="MCTS vs True")
plt.plot(secs_lis,hl_ct,label="Chains vs True")
plt.plot(secs_lis,hl_mc,label="MCTS vs Chains")

plt.title("Hamming Loss Comparison for different times")
plt.xlabel("Seconds")
plt.ylim(0,1)
plt.ylabel("Hamming Loss")
plt.legend()
plt.show();

plt.plot(secs_lis,zo_mt,label="MCTS vs True")
plt.plot(secs_lis,zo_ct,label="Chains vs True")
plt.plot(secs_lis,zo_mc,label="MCTS vs Chains")

plt.title("Zero One Loss Comparison for time different times")
plt.xlabel("Seconds")
plt.ylim(0,1)
plt.ylabel("Zero One Loss")
plt.legend()
plt.show();

## Iteration to time

In [27]:
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.constraints import Constraint
from mcts_inference.policy import UCB

import time

In [28]:
constraint = Constraint(max_iter=True, n_iter=1000)
config = MCTSConfig(n_classes, selection_policy=UCB(), constraint=constraint, verbose=False, step_once=False, parallel=False)

M = 200
while(True):
    t0 = time.time()
    MCTS(chain, X_test[:M], config)
    print(f"{time.time()-t0:.3f}")

KeyboardInterrupt: 

In [22]:
X_test.shape

(2000, 20)

In [53]:
Ns = [100, 500, 1000, 1500, 2000]
Ls = [2,4,8,16,32,64]
avg = 10
M = 100

n_samples = 10000
n_labels = 2
random_state = 0
test_size = 0.2

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
solver = "liblinear"

In [57]:
times = [[[] for _ in Ns] for _ in Ls]

for il, l in enumerate(Ls):
    print("L>", l)
    X, Y = make_multilabel_classification(
        n_samples = n_samples,
        n_features = 4 * l,
        n_classes = l,
        n_labels = n_labels,
        random_state = random_state)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    base = LogisticRegression(solver=solver)
    chain = ClassifierChain(base)

    chain = chain.fit(X_train,Y_train)
    
    for jn, n in enumerate(Ns):
        print("n>", n)
        constraint = Constraint(max_iter=True, n_iter=n)
        config = MCTSConfig(l, selection_policy=UCB(), constraint=constraint, verbose=False, step_once=False, parallel=True)
        for _ in range(avg):
            t0 = time.time()

            MCTS(chain, X_test[:M], config)

            times[il][jn].append(time.time()-t0)

L> 2
n> 100
n> 500
n> 1000
n> 1500
n> 2000
L> 4
n> 100
n> 500


Process SpawnPoolWorker-1337:
Process SpawnPoolWorker-1342:
Process SpawnPoolWorker-1338:
Process SpawnPoolWorker-1343:
Process SpawnPoolWorker-1340:
Process SpawnPoolWorker-1339:


KeyboardInterrupt: 